In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents (1).json


{'intents.json': b'{\n  "intents": [\n    {\n      "tag": "greeting",\n      "patterns": [\n        "Hi",\n\t  "Hi Bot",\n\t  "Hola",\n\t  "Hi, how are you",\n        "How are you",\n        "Is anyone there?",\n        "Hello",\n        "Hey",\n        "What\'s up",\n        "Hi there",\n        "Hey there",\n        "Good morning",\n        "Good afternoon",\n        "Good evening",\n        "Howdy",\n        "Hello there"\n      ],\n      "responses": [\n        "What can i help you with today?",\n        "How can I help you today?"\n      ],\n      "context_set": ""\n    },\n    {\n      "tag": "goodbye",\n      "patterns": [\n        "Bye",\n        "See you later",\n        "Goodbye",\n        "Farewell",\n        "Take care",\n        "Catch you later",\n        "Goodbye for now",\n        "Bye for now",\n        "Talk to you later",\n        "See you soon",\n        "Take it easy",\n        "So long",\n        "Adios",\n        "Good night",\n        "Later",\n        "Have a g

In [ ]:
import gensim.downloader as api
# Load pre-trained GloVe model
model = api.load("glove-wiki-gigaword-100")


[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
import json
import random
import numpy as np
from gensim.models import KeyedVectors
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from tensorflow import keras
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Load the GloVe word vectors
glove_model = KeyedVectors.load_word2vec_format('path_to_glove_file', binary=False)

# Load the intents data
with open("intents.json", "r") as f:
    data = json.load(f)

# Preprocessing
vocab = set()
classes = []
documents = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        pattern = pattern.rstrip("?")
        tokens = word_tokenize(pattern)
        tokens = [token.lower() for token in tokens if token not in set(stopwords.words("english"))]
        vocab.update(tokens)
        documents.append((tokens, intent["tag"]))
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# Generate word embeddings for each token using GloVe
X = []
y = []
for doc in documents:
    vecs = []
    for token in doc[0]:
        if token in glove_model.vocab:
            vecs.append(glove_model[token])  # Get the GloVe word embedding for the token
    if vecs:
        vecs = np.mean(vecs, axis=0)  # Calculate the mean word embedding for the document
        X.append(vecs)
        y.append(classes.index(doc[1]))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Build the model
model1 = Sequential()
model1.add(Dense(128, input_dim=len(X_train[0]), activation="relu"))  # Input layer with ReLU activation
model1.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model1.add(Dense(64, activation="relu"))  # Hidden layer with ReLU activation
model1.add(Dense(len(classes), activation="softmax"))  # Output layer with softmax activation for multi-class classification

model1.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])  # Compile the model

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='auto', restore_best_weights=True)
# Define an early stopping callback to monitor the validation loss during training
# It will stop training if the validation loss doesn't improve for 20 epochs
# verbose=1 prints messages about the early stopping process
# mode='auto' determines the direction of improvement (minimizing loss)
# restore_best_weights=True restores the weights of the model to the ones with the best performance on the validation set

# Train the model
model1.fit(X_train, y_train, epochs=200, validation_split=0.2, batch_size=5, callbacks=early_stop)
# Train the model using the training data
# Perform 200 epochs, use 20% of the training data as the validation set
# Batch size is 5, which means the model will be updated after every 5 samples
# The early_stop callback will monitor the validation loss and stop training if it doesn't improve

# Evaluate the model
loss, accuracy = model1.evaluate(X_test, y_test)
# Evaluate the trained model using the testing data
# Calculate the loss and accuracy of the model on the testing data

print(f"Loss: {loss}, Accuracy: {accuracy}")
# Print the loss and accuracy of the model


Epoch 1/200
37/37 [==============================] - 6s 9ms/step - loss: 2.4307 - accuracy: 0.1366 - val_loss: 2.1763 - val_accuracy: 0.2826
Epoch 2/200
37/37 [==============================] - 0s 5ms/step - loss: 2.1672 - accuracy: 0.2842 - val_loss: 2.0201 - val_accuracy: 0.4130
Epoch 3/200
37/37 [==============================] - 0s 5ms/step - loss: 1.8945 - accuracy: 0.4153 - val_loss: 1.7637 - val_accuracy: 0.5435
Epoch 4/200
37/37 [==============================] - 0s 5ms/step - loss: 1.6371 - accuracy: 0.5246 - val_loss: 1.5147 - val_accuracy: 0.7174
Epoch 5/200
37/37 [==============================] - 0s 5ms/step - loss: 1.4397 - accuracy: 0.5574 - val_loss: 1.2380 - val_accuracy: 0.7174
Epoch 6/200
37/37 [==============================] - 0s 4ms/step - loss: 1.1168 - accuracy: 0.6995 - val_loss: 1.0928 - val_accuracy: 0.7609
Epoch 7/200
37/37 [==============================] - 0s 5ms/step - loss: 1.0883 - accuracy: 0.7541 - val_loss: 0.9878 - val_accuracy: 0.7826
Epoch 8/200
3

In [ ]:
# Vectorize sentence using GloVe embeddings
def vectorize_sentence(sentence, model):
    sentence = sentence.rstrip("?")
    # tokenize the sentence
    sentence_words = nltk.word_tokenize(sentence)
    # lemmatize each word and remove stopwords
    sentence_words = [lemmatizer.lemmatize(word.lower(), pos='v') for word in sentence_words if word not in set(stopwords.words("english"))]
    # create a vector representation
    vecs = []
    for word in sentence_words:
        if word in model:
            vecs.append(model[word])
    if vecs:
        vecs = np.mean(vecs, axis=0)
    else:
        vecs = np.zeros(model.vector_size)
    return vecs


print("Hi there, this is Genie. I'm so happy to chat with you today! 😊")
print("Could you please be more specific with your questions.")
print("I'm always trying to improve my communication skills and learn from your feedback. (type quit to end the conversation):")

while True:
    inp = input("You: ")  # Get user input
    if inp.lower() == "quit":
        # If the user enters "quit", find the "goodbye" intent and print a random response
        for intent in data["intents"]:
            if intent["tag"] == "goodbye":
                print("Bot: " + random.choice(intent["responses"]))
        break  # Exit the loop and end the conversation

    vec = vectorize_sentence(inp, model)  # Vectorize the user input using the GloVe embeddings
    results = model1.predict(np.array([vec]))  # Make a prediction using the trained model
    results_index = np.argmax(results)  # Get the index of the predicted intent
    tag = classes[results_index]

    if results[0][results_index] > 0.5:
        # If the confidence score is above 0.5, find the intent and print a response
        for intent in data["intents"]:
            if intent["tag"] == tag:
                if intent["tag"] == "goodbye" and inp == 'quit':
                    # If the intent is "goodbye" and the user entered "quit", print a goodbye message and end the conversation
                    print("Bot: " + random.choice(intent["responses"]))
                    print("Bot: Goodbye!")
                    break
                if intent["tag"] == "goodbye" and inp != 'quit':
                    # If the intent is "goodbye" but the user entered something other than "quit", print a specific message
                    print("Bot: I am evolving constantly. I apologize that I could not help you with your query. Please give us a call at  and our team will be happy to assist.")
                else:
                    # Print a random response from the intent
                    print("Bot: " + random.choice(intent["responses"]))
                    break
    else:
        # If the confidence score is below 0.5, print a generic message
        print("Bot: I am evolving constantly. I apologize that I could not help you with your query. Please give us a call at  and our team will be happy to assist.")


Hi there, this is Genie. I'm so happy to chat with you today! 😊 
Could you please be more specific with your questions.
I'm always trying to improve my communication skills and learn from your feedback. (type quit to end the conversation):
You: Hi
1/1 [==============================] - 0s 72ms/step
Bot: How can I help you today?
You: togo order
1/1 [==============================] - 0s 18ms/step
Bot: We have many great options to choose from, You can place your order online at https://www.suggioota.com/order-online
You: operational hours
1/1 [==============================] - 0s 21ms/step
Bot: Our hours are 11am-9pm every day, except for Mondays when we're closed. Fridays, we're open from 4pm-9pm.
You: togo available?
1/1 [==============================] - 0s 20ms/step
Bot: I am evolving constantly. I apologize that I could not help you with your query. Please give us a call at  and our team will be happy to assist
You: order food
1/1 [==============================] - 0s 19ms/step
Bot

In [ ]:
path='/content/drive/My Drive/Colab Notebooks' #Declare the path where the code and model file has been placed

In [ ]:
model1.save('Genie_word2vec.h5')#save the model